In [59]:
from pathlib import Path
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import soundfile as sf
import pandas as pd

import os 
import itertools
import re
import math
import csv
import random
from sklearn import tree
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from scipy.fftpack import fftn, ifftn, fft, ifft
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy import signal
import pickle
from sklearn.externals import joblib
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import ShuffleSplit
import warnings

warnings.filterwarnings('ignore') 
#df = pd.read_csv("features/Features10too200hz.csv")

#df = pd.read_csv("features/unfilteredFeatures.csv")
#df = pd.read_csv("featuress/allEmgData10to200hz_noAudible.csv")
df = pd.read_csv("featuress/allEmgData_noAudible_6Channel.csv")
#df = pd.read_csv("featuress/allEmgData10too200hz_noAudible_window.csv")
df = df.drop ('channel_5_mav', axis=1)
df = df.drop ('channel_5_mav2', axis=1)
df = df.drop ('channel_5_rms', axis=1)
df = df.drop ('channel_5_iav', axis=1)
df = df.drop ('channel_5_ssi', axis=1)
df = df.drop ('channel_5_var', axis=1)
df = df.drop ('channel_5_wl', axis=1)
df = df.drop ('channel_5_iemg', axis=1)
df = df.drop ('channel_5_aac', axis=1)
df = df.drop ('channel_5_zc', axis=1)
df = df.drop ('channel_5_ssc', axis=1)
df = df.drop ('channel_5_wamp', axis=1)
df = df.drop ('channel_5_medf', axis=1)
df = df.drop ('channel_5_meanf', axis=1)
#df = pd.read_csv("Features_all_no_audible.csv")
#df = df[(df.modeID != 'audible')]
#df= df[df.sessionID == 4002 ]
df

,channel_1_mav,channel_1_mav2,channel_1_rms,channel_1_iav,channel_1_ssi,channel_1_var,channel_1_wl,channel_1_iemg,channel_1_aac,channel_1_zc,...,channel_6_aac,channel_6_zc,channel_6_ssc,channel_6_wamp,channel_6_medf,channel_6_meanf,userID,sessionID,uttID,modeID
0,0.067106,0.067103,0.081097,224.738037,22.025275,0.003286,191.597839,224.738037,0.057210,682,...,0.051220,374,1713,6,106.306306,115.891113,1,1001,10010001,audible
1,0.063718,0.063648,0.075014,143.621399,12.683361,0.002283,103.957947,143.621399,0.046122,288,...,0.039666,128,1199,3,100.446429,114.028062,1,1001,10010002,audible
2,0.067169,0.067090,0.081422,272.503754,26.896239,0.003316,227.992035,272.503754,0.056197,816,...,0.057614,520,2043,15,112.871287,119.874289,1,1001,10010003,audible
3,0.065079,0.065007,0.078301,271.053162,25.535573,0.002825,224.132660,271.053162,0.053813,772,...,0.064233,596,2124,21,106.265060,117.951279,1,1001,10010004,audible
4,0.065251,0.065301,0.077894,306.745789,28.522983,0.002760,239.254150,306.745789,0.050894,756,...,0.058570,614,2374,18,116.417910,121.944233,1,1001,10010005,audible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,0.105939,0.105920,0.114247,247.473846,30.490382,0.002267,61.923187,247.473846,0.026508,70,...,0.006785,0,1259,0,0.000000,90.523269,8,8019,80190246,whisper
4796,0.103561,0.103695,0.107324,197.284485,21.942521,0.000830,36.726807,197.284485,0.019279,10,...,0.005655,2,1043,0,0.000000,80.479425,8,8019,80190247,whisper
4797,0.103896,0.103751,0.108278,234.181610,26.425943,0.000987,47.587830,234.181610,0.021113,20,...,0.005838,2,1205,0,0.000000,81.305444,8,8019,80190248,whisper
4798,0.103268,0.103307,0.106641,176.794922,19.469253,0.000728,32.907990,176.794922,0.019222,6,...,0.005437,0,960,0,0.000000,70.628165,8,8019,80190249,whisper


In [60]:
def split_data_per_session(session_id):
    temp_df = df[(df.sessionID == session_id)]
    data = temp_df.copy()
    data = shuffle(data)
    y = data['modeID']
    data.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return train_test_split(data, y, test_size=0.2)

def split_data_session(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['sessionID'].isin(train_user_ids)]
    test_df = data.loc[data['sessionID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

def split_data_utterance(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['userID'].isin(train_user_ids)]
    test_df = data.loc[data['userID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

def split_data(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['userID'].isin(train_user_ids)]
    test_df = data.loc[data['userID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

In [61]:
def lda_user1_session1():
    train_ids = [1002,1003]
    test_ids = [1001]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 1 is "+str(score))
    return score
    
def lda_user1_session2():
    train_ids = [1001,1003]
    test_ids = [1002]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 2 is "+str(score))
    return score
    
def lda_user1_session3():
    train_ids = [1001,1002]
    test_ids = [1003]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 3 is "+str(score))
    return score
    
def lda_user2_session1():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2001]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 1 is "+str(score))
    return score  
    
def lda_user2_session3():
    train_ids = [2001, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2003]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 3 is "+str(score))
    return score 
    
def lda_user2_session4():
    train_ids = [2003, 2001, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2004]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 4 is "+str(score))
    return score    
    
def lda_user2_session5():
    train_ids = [2003, 2004, 2001, 2006, 2007, 2008, 2009, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2005]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 5 is "+str(score))
    return score  
    
def lda_user2_session6():
    train_ids = [2003, 2004, 2005, 2001, 2007, 2008, 2009, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2006]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 6 is "+str(score))
    return score   
    
def lda_user2_session7():
    train_ids = [2003, 2004, 2005, 2006, 2001, 2008, 2009, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2007]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 7 is "+str(score))
    return score     
    
def lda_user2_session8():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2001, 2009, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2008]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 8 is "+str(score))
    return score 
    
def lda_user2_session9():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2001, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2009]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 9 is "+str(score))
    return score   
    
def lda_user2_session10():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2001, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2010]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 10 is "+str(score))
    return score     
    
def lda_user2_session12():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2001, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2012]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 12 is "+str(score))
    return score 
    
def lda_user2_session13():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2001,
       2028, 2029, 2031, 2032] 
    test_ids = [2013]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 13 is "+str(score))
    return score   
    
def lda_user2_session28():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2001,
       2013, 2029, 2031, 2032] 
    test_ids = [2028]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 28 is "+str(score))
    return score  
    
def lda_user2_session29():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2001,
       2028, 2013, 2031, 2032] 
    test_ids = [2029]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 29 is "+str(score))
    return score     
    
def lda_user2_session31():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2001,
       2028, 2029, 2013, 2032] 
    test_ids = [2031]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 31 is "+str(score))
    return score  
    
def lda_user2_session32():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2001,
       2028, 2029, 2031, 2013] 
    test_ids = [2032]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 32 is "+str(score))
    return score 
    
def lda_user4_session1():
    train_ids = [4002]
    test_ids = [4001]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 1 is "+str(score))
    return score   
    
def lda_user4_session2():
    train_ids = [4001]
    test_ids = [4002]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 2 is "+str(score))
    return score 
    
def lda_user7_session1():
    train_ids = [7002]
    test_ids = [7001]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 1 is "+str(score))
    return score 
    
def lda_user7_session2():
    train_ids = [7001]
    test_ids = [7002]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 2 is "+str(score))
    return score 
    
def lda_user8_session2():
    train_ids = [8003, 8010, 8016, 8017, 8018, 8019]
    test_ids = [8002]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 2 is "+str(score))
    return score   
    
def lda_user8_session3():
    train_ids = [8002, 8010, 8016, 8017, 8018, 8019]
    test_ids = [8003]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 3 is "+str(score))
    return score  
    
def lda_user8_session10():
    train_ids = [8003, 8002, 8016, 8017, 8018, 8019]
    test_ids = [8010]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 10 is "+str(score))
    return score   
    
def lda_user8_session16():
    train_ids = [8003, 8010, 8002, 8017, 8018, 8019]
    test_ids = [8016]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 16 is "+str(score))
    return score    
    
def lda_user8_session17():
    train_ids = [8003, 8010, 8016, 8002, 8018, 8019]
    test_ids = [8017]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 17 is "+str(score))
    return score  
    
def lda_user8_session18():
    train_ids = [8003, 8010, 8016, 8017, 8002, 8019]
    test_ids = [8018]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 18 is "+str(score))
    return score     
    
def lda_user8_session19():
    train_ids = [8003, 8010, 8016, 8017, 8018, 8002]
    test_ids = [8019]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score 

In [62]:
def lda():
    ids = [1001,1002,1003,2001,2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2013,
       2028, 2029, 2031, 2032,4001,4002,7001,7002,8002,8003, 8010, 8016, 8017, 8018, 8019]
    #ids = [ 8010, 8016, 8017, 8018, 8019]
    scores = np.array([])
    for x in range (len(ids)):
        test_ids = []
        test_ids = ids[x:x+1]
        train_ids = np.delete(ids,x)
        #print (test_ids)
        X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
        k_clf = LinearDiscriminantAnalysis()
        k_clf.fit(X_train, y_train)
        score = k_clf.score(X_test, y_test)
        sscore = np.array([score])
        scores = np.concatenate([scores,sscore],axis=0)
        a=np.unique(k_clf.predict(X_test))
        plot_confusion_matrix(k_clf, X_test, y_test,normalize='true')  # doctest: +SKIP
        plt.title("User" + str(test_ids[0]))
        #plt.savefig(os.path.join("User" + str(test_ids[0])+".png"), dpi=600, format='png', bbox_inches='tight')
        plt.show()
        print("Accuracy for session " + str(test_ids) +" is "+str(score))
    print ("Durchschnitt: ")
    print((scores.mean()))

In [63]:
def lda_all():
    print ("User 1 : " )
    a = lda_user1_session1()
    b = lda_user1_session2()
    c = lda_user1_session3()
    print ("User 2 : " )
    d = lda_user2_session1()
    e = lda_user2_session3()
    f = lda_user2_session4()
    g = lda_user2_session5()
    h = lda_user2_session6()
    i = lda_user2_session7()
    j = lda_user2_session8()
    k = lda_user2_session9()
    l = lda_user2_session10()
    m = lda_user2_session12()
    n = lda_user2_session13()
    o = lda_user2_session28()
    p = lda_user2_session29()
    q = lda_user2_session31()
    r = lda_user2_session32()
    print ("User 4 : " )
    s = lda_user4_session1()
    t = lda_user4_session2()
    print ("User 7 : " )
    u = lda_user7_session1()
    v = lda_user7_session2()
    print ("User 8 : " )
    w = lda_user8_session2()
    x = lda_user8_session3()
    y = lda_user8_session10()
    z = lda_user8_session16()
    aa = lda_user8_session17()
    bb = lda_user8_session18()
    cc = lda_user8_session19()
    scores = np.array([a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z,aa,bb,cc])
    print("Durchschnitt:")
    print((scores.mean()))
    print(scores.std())


In [44]:
lda_all() 

User 1 : 
Accuracy for session 1 is 0.68
Accuracy for session 2 is 0.84
Accuracy for session 3 is 0.66
User 2 : 
Accuracy for session 1 is 0.6066666666666667
Accuracy for session 3 is 0.74
Accuracy for session 4 is 0.44
Accuracy for session 5 is 0.8666666666666667
Accuracy for session 6 is 0.7933333333333333
Accuracy for session 7 is 0.8266666666666667
Accuracy for session 8 is 0.8133333333333334
Accuracy for session 9 is 0.6333333333333333
Accuracy for session 10 is 0.76
Accuracy for session 12 is 0.7933333333333333
Accuracy for session 13 is 0.24
Accuracy for session 28 is 0.5066666666666667
Accuracy for session 29 is 0.37333333333333335
Accuracy for session 31 is 0.5733333333333334
Accuracy for session 32 is 0.5866666666666667
User 4 : 
Accuracy for session 1 is 0.22
Accuracy for session 2 is 0.32
User 7 : 
Accuracy for session 1 is 0.36
Accuracy for session 2 is 0.38
User 8 : 
Accuracy for session 2 is 0.5666666666666667
Accuracy for session 3 is 0.62
Accuracy for session 10 is 0.4

In [51]:
lda_all() #ohne 6

User 1 : 
Accuracy for session 1 is 0.68
Accuracy for session 2 is 0.8466666666666667
Accuracy for session 3 is 0.7266666666666667
User 2 : 
Accuracy for session 1 is 0.68
Accuracy for session 3 is 0.7533333333333333
Accuracy for session 4 is 0.43333333333333335
Accuracy for session 5 is 0.8466666666666667
Accuracy for session 6 is 0.7933333333333333
Accuracy for session 7 is 0.8066666666666666
Accuracy for session 8 is 0.8
Accuracy for session 9 is 0.6666666666666666
Accuracy for session 10 is 0.76
Accuracy for session 12 is 0.78
Accuracy for session 13 is 0.23333333333333334
Accuracy for session 28 is 0.5
Accuracy for session 29 is 0.46
Accuracy for session 31 is 0.6066666666666667
Accuracy for session 32 is 0.66
User 4 : 
Accuracy for session 1 is 0.31333333333333335
Accuracy for session 2 is 0.3333333333333333
User 7 : 
Accuracy for session 1 is 0.3466666666666667
Accuracy for session 2 is 0.34
User 8 : 
Accuracy for session 2 is 0.5933333333333334
Accuracy for session 3 is 0.62
Ac

In [64]:
lda_all() #ohne 5 und 6

User 1 : 
Accuracy for session 1 is 0.72
Accuracy for session 2 is 0.8333333333333334
Accuracy for session 3 is 0.7466666666666667
User 2 : 
Accuracy for session 1 is 0.6466666666666666
Accuracy for session 3 is 0.6666666666666666
Accuracy for session 4 is 0.4066666666666667
Accuracy for session 5 is 0.8533333333333334
Accuracy for session 6 is 0.8333333333333334
Accuracy for session 7 is 0.8066666666666666
Accuracy for session 8 is 0.8133333333333334
Accuracy for session 9 is 0.6866666666666666
Accuracy for session 10 is 0.7533333333333333
Accuracy for session 12 is 0.74
Accuracy for session 13 is 0.26666666666666666
Accuracy for session 28 is 0.6466666666666666
Accuracy for session 29 is 0.5533333333333333
Accuracy for session 31 is 0.6866666666666666
Accuracy for session 32 is 0.5333333333333333
User 4 : 
Accuracy for session 1 is 0.09333333333333334
Accuracy for session 2 is 0.32
User 7 : 
Accuracy for session 1 is 0.36666666666666664
Accuracy for session 2 is 0.34
User 8 : 
Accura

### from sklearn.metrics import plot_confusion_matrix
from sklearn.externals import joblib
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
def lda_allss():
    y_pred =[]
    y_actu=[]
    labels = ['audible', 'silent','whisper']
    print ("User 1 : " )
    y_p,y_a = lda_user1_session1()
    y_pred =np.concatenate((y_pred, y_p))
    y_actu =np.concatenate((y_actu, y_a))
    y_p,y_a = lda_user1_session2()
    y_pred =np.concatenate((y_pred, y_p))
    y_actu =np.concatenate((y_actu, y_a))
    y_p,y_a = lda_user1_session3()
    y_pred =np.concatenate((y_pred, y_p))
    y_actu =np.concatenate((y_actu, y_a))
    print ("User 2 : " )
    y_p,y_a = lda_user2_session1()
    y_pred =np.concatenate((y_pred, y_p))
    y_actu =np.concatenate((y_actu, y_a))
    y_p,y_a = lda_user2_session3()
    y_pred =np.concatenate((y_pred, y_p))
    y_actu =np.concatenate((y_actu, y_a))
    y_p,y_a = lda_user2_session4()
    y_pred =np.concatenate((y_pred, y_p))
    y_actu =np.concatenate((y_actu, y_a))
    y_p,y_a = lda_user2_session5()
    y_pred =np.concatenate((y_pred, y_p))
    y_actu =np.concatenate((y_actu, y_a))
    y_p,y_a = lda_user2_session6()
    y_pred =np.concatenate((y_pred, y_p))
    y_actu =np.concatenate((y_actu, y_a))
    y_p,y_a = lda_user2_session7()
    y_pred =np.concatenate((y_pred, y_p))
    y_actu =np.concatenate((y_actu, y_a))
    y_p,y_a = lda_user2_session8()
    y_pred =np.concatenate((y_pred, y_p))
    y_actu =np.concatenate((y_actu, y_a))
    y_p,y_a = lda_user2_session9()
    y_pred =np.concatenate((y_pred, y_p))
    y_actu =np.concatenate((y_actu, y_a))
    y_p,y_a = lda_user2_session10()
    y_pred =np.concatenate((y_pred, y_p))
    y_actu =np.concatenate((y_actu, y_a))
    y_p,y_a = lda_user2_session12()
    y_pred =np.concatenate((y_pred, y_p))
    y_actu =np.concatenate((y_actu, y_a))
    y_p,y_a = lda_user2_session13()
    y_pred =np.concatenate((y_pred, y_p))
    y_actu =np.concatenate((y_actu, y_a))
    y_p,y_a = lda_user2_session28()
    y_pred =np.concatenate((y_pred, y_p))
    y_actu =np.concatenate((y_actu, y_a))
    y_p,y_a = lda_user2_session29()
    y_pred =np.concatenate((y_pred, y_p))
    y_actu =np.concatenate((y_actu, y_a))
    y_p,y_a = lda_user2_session31()
    y_pred =np.concatenate((y_pred, y_p))
    y_actu =np.concatenate((y_actu, y_a))
    y_p,y_a = lda_user2_session32()
    y_pred =np.concatenate((y_pred, y_p))
    y_actu =np.concatenate((y_actu, y_a))
    print ("User 4 : " )
    y_p,y_a = lda_user4_session1()
    y_pred =np.concatenate((y_pred, y_p))
    y_actu =np.concatenate((y_actu, y_a))
    y_p,y_a = lda_user4_session2()
    y_pred =np.concatenate((y_pred, y_p))
    y_actu =np.concatenate((y_actu, y_a))
    print ("User 7 : " )
    y_p,y_a = lda_user7_session1()
    y_pred =np.concatenate((y_pred, y_p))
    y_actu =np.concatenate((y_actu, y_a))
    y_p,y_a = lda_user7_session2()
    y_pred =np.concatenate((y_pred, y_p))
    y_actu =np.concatenate((y_actu, y_a))
    print ("User 8 : " )
    y_p,y_a = lda_user8_session2()
    y_pred =np.concatenate((y_pred, y_p))
    y_actu =np.concatenate((y_actu, y_a))
    y_p,y_a = lda_user8_session3()
    y_pred =np.concatenate((y_pred, y_p))
    y_actu =np.concatenate((y_actu, y_a))
    y_p,y_a = lda_user8_session10()
    y_pred =np.concatenate((y_pred, y_p))
    y_actu =np.concatenate((y_actu, y_a))
    y_p,y_a = lda_user8_session16()
    y_pred =np.concatenate((y_pred, y_p))
    y_actu =np.concatenate((y_actu, y_a))
    y_p,y_a = lda_user8_session17()
    y_pred =np.concatenate((y_pred, y_p))
    y_actu =np.concatenate((y_actu, y_a))
    y_p,y_a = lda_user8_session18()
    y_pred =np.concatenate((y_pred, y_p))
    y_actu =np.concatenate((y_actu, y_a))
    y_p,y_a = lda_user8_session19()
    y_pred =np.concatenate((y_pred, y_p))
    y_actu =np.concatenate((y_actu, y_a))
    a = confusion_matrix(y_actu, y_pred, labels=labels,normalize='true')
    plt.figure()
    cm_display = ConfusionMatrixDisplay(a,display_labels=labels).plot()
    plt.title("Speechmode")
    plt.savefig(os.path.join("AllSessionModeConfMatUnf.png"), dpi=600, format='png', bbox_inches='tight')
    #scores = np.array([a,b,c,d,e,f,g,h,i,j,k,l,m,o,p,p,q,r,s,t,u,v,w,x,y,z,aa,bb,cc])
    #print("Durchschnitt:")
    #print((scores.mean()))
    #print(scores.std())


In [19]:
lda_all()

User 1 : 


NameError: name 'RandomForestClassifier' is not defined